## Fake News Corpus
### https://huggingface.co/datasets/andyP/fake_news_en_opensources
Mapping: (type)  ->  label
- (satire)      ->  1-"Satire"
- (fake)  ->  2-"Hoax"
- (bias)        ->  3-"Propaganda"
- (reliable)    ->  4-"Reliable News"

In [9]:
import sys
import csv
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler

In [10]:
csv.field_size_limit(500000)

500000

In [11]:
CHUNK_SIZE = 10 ** 6

In [12]:
with pd.read_csv('opensources_fake_news_cleaned.csv', chunksize=CHUNK_SIZE, index_col="id") as reader:
    for chunk in reader:
        type_content = chunk[['type', 'content']]
        filtered = type_content[(type_content.type == 'satire') | (type_content.type == 'fake') | (type_content.type == 'bias') | (type_content.type == 'reliable')]
        filtered['type'] = filtered['type'].map({'satire': 1, 'fake': 2, 'bias': 3, 'reliable': 4})
        filtered.to_csv('fakenewscorpus.csv', mode='a', index=False, header=False)

/var/folders/bc/p9431k390vddqs7hq2n8vrlh0000gn/T/ipykernel_1333/2384829192.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['type'] = filtered['type'].map({'satire': 1, 'fake': 2, 'bias': 3, 'reliable': 4})
/var/folders/bc/p9431k390vddqs7hq2n8vrlh0000gn/T/ipykernel_1333/2384829192.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['type'] = filtered['type'].map({'satire': 1, 'fake': 2, 'bias': 3, 'reliable': 4})
/var/folders/bc/p9431k390vddqs7hq2n8vrlh0000gn/T/ipykernel_1333/2384829192

In [ ]:
# df = pd.read_csv('fakenewscorpus.csv', header=None, index_col=False)
y = np.empty(0)
X = np.empty(0)

# Iterate over chunks
for chunk in pd.read_csv('fakenewscorpus.csv', header=None, chunksize=CHUNK_SIZE):
    y_chunk = chunk.iloc[:, 0].values  
    X_chunk = chunk.iloc[:, 1].values  
    
    y = np.concatenate((y, y_chunk))
    X = np.concatenate((X, X_chunk))

: 

In [ ]:
# X = df.iloc[:, 1]
# y = df.iloc[:, 0]

print(X.head())
print(y.value_counts())

0    FAIR USE NOTICE. Many of the stories on this s...
1    From Conservapedia\n\nPsychology is the scient...
2    From Conservapedia\n\nLeVeyan Satanism is a fo...
3    Robert Parry| Commondreams\n\nIn the heat of t...
4    Two weeks ago, when discussing the troubles pl...
Name: 1, dtype: object
0
4    1807323
3     769874
2     494184
1      84735
Name: count, dtype: int64


In [ ]:
rus = RandomUnderSampler(sampling_strategy={1: 15000, 2: 15000, 3: 15000, 4: 15000})
X_undersampled, y_undersampled  = rus.fit_resample(X.to_frame(), y.to_frame())

In [ ]:
print(X_undersampled.head())
print(y_undersampled.value_counts())

                                                         1
2512605  DOGS generally fall out within less than half ...
336352   Embarrassment reigned at the American Israel P...
2926131  #Donald Trump was the underdog as he trailed t...
1850024  HUNDREDS of thousands of people descended onto...
887632   WASHINGTON—In an effort to streamline the proc...
0
1    15000
2    15000
3    15000
4    15000
Name: count, dtype: int64


In [ ]:
df_undersampled = pd.concat([y_undersampled, X_undersampled], axis=1)

In [ ]:
X_undersampled = df_undersampled.iloc[:, 1]
y_undersampled = df_undersampled.iloc[:, 0]

print(X_undersampled.head())
print(y_undersampled.value_counts())

2512605    DOGS generally fall out within less than half ...
336352     Embarrassment reigned at the American Israel P...
2926131    #Donald Trump was the underdog as he trailed t...
1850024    HUNDREDS of thousands of people descended onto...
887632     WASHINGTON—In an effort to streamline the proc...
Name: 1, dtype: object
0
1    15000
2    15000
3    15000
4    15000
Name: count, dtype: int64


In [ ]:
df_undersampled.to_csv('opensources_fakenewscorpus_balancedtest.csv', index=False, header=False)